In [3]:
#importing libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [5]:
#training and testing directory
train_dir = pd.read_csv('../input/fashionmnist/fashion-mnist_train.csv')
test_dir = pd.read_csv('../input/fashionmnist/fashion-mnist_test.csv')

In [6]:
train_dir.head()

In [7]:
#defining train labels and train sets
train_label = np.array(train_dir['label'])
train_ds = np.array(train_dir.drop(['label'], axis = 1))

#defining test labels and test sets
test_label = np.array(test_dir['label'])
test_ds = np.array(test_dir.drop(['label'], axis = 1))

In [8]:
#feature scaling
train_ds = train_ds/255.

#reshaping for the input of neural network(shape of input image 28x28)
train_ds = train_ds.reshape(-1,28,28,1)

#feature scaling
test_ds = test_ds/255.

#reshaping for the input of neural network
test_ds = test_ds.reshape(-1,28,28,1)

In [9]:
#test labels and training labels are in range(0,9),hence applying onehotecoding()
from tensorflow.keras.utils import to_categorical
train_label = to_categorical(train_label,10)
test_label = to_categorical(test_label,10)

In [11]:
#classes of training set
names = ['T_shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

plt.figure(figsize=(7, 5))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_ds[i])
    plt.title(names[train_label[i].argmax()])
    plt.tight_layout()
plt.show()

In [12]:
from keras.layers import Conv2D,Dense,MaxPool2D,Flatten,Dropout

#defining the model as a sequence of layers.
model = keras.models.Sequential()

#adding 1st Convolutional layer
model.add(Conv2D(filters = 32,kernel_size = 3,activation = 'relu' ,input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=2))
model.add(Dropout(0.2))

#flattening the output
model.add(Flatten())

#adding dense layer
model.add(Dense(64,activation = "relu"))
model.add(Dropout(0.3))

#adding final layer
model.add(Dense(10,activation = "softmax"))

#compiling the model
model.compile(optimizer=keras.optimizers.Adam(lr=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])

#training the model
cnn = model.fit(train_ds,train_label, epochs=20,batch_size = 32 ,validation_split=0.1)

In [13]:
#summary of model
model.summary()

In [14]:
#evaluating model on test set
model.evaluate(test_ds,test_label)

In [15]:
#storing predictions in variable(pred)
pred=model.predict(test_ds,verbose=1)

In [16]:
#saving index of maximum value of pred in preds
preds = []
for i in range(0,len(pred)):
    preds.append(pred[i].argmax())

#saving index of maximum value of training set in actual
actual = []
for i in range(0,len(test_label)):
    actual.append(test_label[i].argmax())

In [17]:
#classification report
from sklearn import metrics
report = metrics.classification_report(actual, preds, target_names=names)
print(report)

In [18]:
#graph for accuracy
plt.plot(cnn.history['accuracy'])
plt.plot(cnn.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

#graph for loss
plt.plot(cnn.history['loss'])
plt.plot(cnn.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()